## Data Wrangling

In this code, we generate the aggregated records (formation period records, role period records, and role assignment records) used in formation clustering, role labeling, and other applications from FGP files.

Unfortunately, since we only share the single match data (ID 17985) due to our company's security issue, the following code cannot regenerate the above records when you only use the shared data, i.e. `17985.ugp`. (That is, you can only check how these records are generated by reading this code.)

Instead, we directly share the `form_periods.pkl`, `role_periods.csv`, and `role_records.csv` (resulting files of applying this code to the entire dataset) so that you can reproduce the results of aforementioned tasks in another code files.

In [4]:
import os
wd = os.path.normpath(os.getcwd() + '/..')
os.chdir(wd)
os.getcwd()

'/home/hyunsung/Workspace/soccercpd'

In [5]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
from joblib import Parallel, delayed
from src.myconstants import *

pd.set_option('display.width', 250)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Merging formation period records

In [6]:
mode = 'kernel_cosine'
form_periods_files = os.listdir(f'data/{mode}/form')
form_periods_files.sort()
form_periods_list = []

for file in form_periods_files:
    path = f'data/{mode}/form/{file}'
    form_periods_list.append(pd.read_pickle(path))

form_periods = pd.concat(form_periods_list, ignore_index=True)
form_periods.to_pickle(f'data/{mode}/form_periods.pkl')
form_periods

,activity_id,form_period,session,start_dt,end_dt,duration,coords,edge_mat
0,12864,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820.0,"[[-706.0, 1597.0], [-1362.0, 686.0], [910.0, 1...","[[0.0, 0.981, 0.953, 0.205, 0.027, 0.814, 0.27..."
1,12868,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820.0,"[[-807.0, -1612.0], [44.0, 199.0], [-373.0, -2...","[[0.0, 0.31, 0.721, 0.997, 0.207, 0.049, 0.115..."
2,12868,2,2,2020-01-01 15:02:00,2020-01-01 15:50:00,2880.0,"[[-928.0, -1910.0], [-111.0, 658.0], [-206.0, ...","[[0.0, 0.168, 0.853, 0.989, 0.126, 0.042, 0.24..."
3,12870,1,1,2020-01-01 13:30:00,2020-01-01 14:19:00,2940.0,"[[-1400.0, 57.0], [439.0, -1071.0], [-283.0, 3...","[[0.0, 0.219, 0.884, 0.097, 0.395, 0.959, 0.34..."
4,12870,2,2,2020-01-01 14:33:00,2020-01-01 15:23:00,3000.0,"[[-1266.0, -146.0], [292.0, -877.0], [-205.0, ...","[[0.0, 0.393, 0.871, 0.067, 0.217, 0.998, 0.28..."
...,...,...,...,...,...,...,...,...
878,9064,3,2,2020-01-01 16:27:10,2020-01-01 16:51:00,1430.0,"[[-1538.0, 511.0], [-1621.0, -701.0], [-0.0, -...","[[0.0, 0.984, 0.493, 0.047, 0.01, 0.025, 0.836..."
879,9281,1,1,2020-01-01 15:00:00,2020-01-01 15:47:00,2820.0,"[[-1437.0, 113.0], [-1109.0, -959.0], [1523.0,...","[[0.0, 0.969, 0.053, 0.032, 0.028, 0.201, 0.98..."
880,9281,2,2,2020-01-01 16:03:00,2020-01-01 16:51:00,2880.0,"[[-1253.0, -90.0], [-1090.0, -1033.0], [1471.0...","[[0.0, 0.933, 0.042, 0.034, 0.061, 0.195, 0.97..."
881,9285,1,1,2020-01-01 14:01:00,2020-01-01 14:48:00,2820.0,"[[-638.0, 1741.0], [-1425.0, 599.0], [-1561.0,...","[[0.0, 0.986, 0.243, 0.315, 0.159, 0.738, 0.23..."


### Generating role period records

In [7]:
role_period_cols = [
    LABEL_ACTIVITY_ID, LABEL_PLAYER_PERIOD, LABEL_FORM_PERIOD, LABEL_ROLE_PERIOD,
    LABEL_SESSION, LABEL_START_TIME, LABEL_END_TIME
]
activity_records = pd.read_csv('data/activity_records.csv', header=0, encoding='utf-8-sig')
form_periods = pd.read_pickle(f'data/{mode}/form_periods.pkl')

In [8]:
def generate_role_period_records(fgp_file, mode='gseg_avg'):
    activity_id = int(fgp_file.split('.')[0])
    fgp_path = f'data/{mode}/fgp/{fgp_file}'
    fgp_df = pd.read_csv(fgp_path, header=0, encoding='utf-8-sig')

    grouped = fgp_df.groupby(LABEL_ROLE_PERIOD)
    match_role_periods = grouped[[LABEL_PLAYER_PERIOD, LABEL_FORM_PERIOD, LABEL_SESSION]].first()
    match_role_periods[LABEL_ACTIVITY_ID] = activity_id
    match_role_periods[LABEL_START_TIME] = grouped[LABEL_GAMETIME].min().apply(lambda x: int(x[:2]))
    match_role_periods[LABEL_END_TIME] = grouped[LABEL_GAMETIME].max().apply(lambda x: int(x[:2]))
    match_role_periods.reset_index(inplace=True)

    return match_role_periods[role_period_cols]

In [9]:
fgp_files = os.listdir(f'data/{mode}/fgp')
role_period_list = Parallel(n_jobs=50)(
    delayed(generate_role_period_records)(f, mode) for f in tqdm(fgp_files)
)
role_periods = pd.concat(role_period_list, axis=0, ignore_index=True).astype(int)
role_periods.sort_values([LABEL_ACTIVITY_ID, LABEL_ROLE_PERIOD], inplace=True)
role_periods

100%|██████████| 456/456 [00:04<00:00, 109.35it/s]


,activity_id,player_period,form_period,role_period,session,start_time,end_time
801,1879,1,1,1,1,0,35
802,1879,2,1,2,1,35,47
803,1879,3,2,3,2,0,31
804,1879,4,2,4,2,31,42
805,1879,4,2,5,2,42,49
...,...,...,...,...,...,...,...
1890,22875,3,2,4,2,0,18
1891,22875,4,2,5,2,18,24
1892,22875,4,2,6,2,24,37
1893,22875,4,2,7,2,37,44


In [10]:
role_periods.to_csv(f'data/{mode}/role_periods.csv', index=False)

### Generating and merging role assignment records

In [11]:
role_record_cols = [
    LABEL_PLAYER_PERIOD, LABEL_FORM_PERIOD, LABEL_SESSION, LABEL_DATETIME, LABEL_BASE_ROLE
]

In [12]:
fgp_path = f'data/{mode}/fgp/1879.csv'
pd.read_csv(fgp_path, header=0, encoding='utf-8-sig')

,player_id,squad_num,player_name,datetime,session,gametime,player_period,form_period,role_period,x,y,x_norm,y_norm,role,base_role,switch_rate
0,1248,7,P07,2020-01-01 16:35:01,1.0,35:01.0,2.0,1,2,5697.0,7189.0,4344.1,3496.4,3,3,0.7
1,1248,7,P07,2020-01-01 16:35:02,1.0,35:02.0,2.0,1,2,5551.0,7142.0,4220.3,3445.6,3,3,0.7
2,1248,7,P07,2020-01-01 16:35:03,1.0,35:03.0,2.0,1,2,5421.0,7088.0,4111.6,3389.6,3,3,0.7
3,1248,7,P07,2020-01-01 16:35:04,1.0,35:04.0,2.0,1,2,5282.0,7015.0,4001.3,3327.3,3,3,0.7
4,1248,7,P07,2020-01-01 16:35:05,1.0,35:05.0,2.0,1,2,5149.0,6945.0,3890.2,3277.2,3,3,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58540,1236,42,P42,2020-01-01 17:32:56,2.0,30:56.0,3.0,2,3,8700.0,5629.0,462.4,1111.9,10,10,0.9
58541,1236,42,P42,2020-01-01 17:32:57,2.0,30:57.0,3.0,2,3,8712.0,5497.0,461.5,995.2,10,10,0.9
58542,1236,42,P42,2020-01-01 17:32:58,2.0,30:58.0,3.0,2,3,8715.0,5351.0,458.7,865.8,10,10,0.9
58543,1236,42,P42,2020-01-01 17:32:59,2.0,30:59.0,3.0,2,3,8730.0,5216.0,472.4,741.0,10,10,0.9


In [13]:
def generate_role_records(fgp_file, mode='gseg_avg', freq=1):
    activity_id = int(fgp_file.split('.')[0])
    fgp_path = f'data/{mode}/fgp/{fgp_file}'
    fgp_df = pd.read_csv(fgp_path, header=0, encoding='utf-8-sig')
    fgp_df[LABEL_DATETIME] = fgp_df[LABEL_DATETIME].apply(lambda dt: datetime.strptime(dt, '%Y-%m-%d %H:%M:%S'))

    grouped = fgp_df.groupby(HEADER_ROSTER + [LABEL_ROLE_PERIOD], as_index=False)
    role_records = grouped[role_record_cols].first()
    role_records[LABEL_ACTIVITY_ID] = activity_id
    role_records[LABEL_START_DT] = grouped[LABEL_DATETIME].first()[LABEL_DATETIME] - timedelta(seconds=freq)
    role_records[LABEL_END_DT] = grouped[LABEL_DATETIME].last()[LABEL_DATETIME]
    role_records[LABEL_DURATION] = grouped[LABEL_GAMETIME].count()[LABEL_GAMETIME] * freq

    role_records = pd.merge(role_records, form_periods[[LABEL_ACTIVITY_ID, LABEL_FORM_PERIOD, LABEL_COORDS]])
    role_records[LABEL_X] = role_records.apply(lambda x: x[LABEL_COORDS][x[LABEL_BASE_ROLE]-1, 0], axis=1)
    role_records[LABEL_Y] = role_records.apply(lambda x: x[LABEL_COORDS][x[LABEL_BASE_ROLE]-1, 1], axis=1)

    role_records = role_records[HEADER_ROLE_RECORDS].sort_values(
        [LABEL_PLAYER_ID, LABEL_ROLE_PERIOD], ignore_index=True)
    target_path = f'data/{mode}/role/{activity_id}.csv'
    role_records.to_csv(target_path, index=False, encoding='utf-8-sig')

    return target_path

In [14]:
target_dir = f'data/{mode}/role'
if not os.path.exists(target_dir):
    os.mkdir(target_dir)

fgp_files = os.listdir(f'data/{mode}/fgp')
target_paths = Parallel(n_jobs=50)(delayed(generate_role_records)(f, mode) for f in tqdm(fgp_files))
target_paths.sort()

pd.read_csv(target_paths[0], header=0)

100%|██████████| 456/456 [00:09<00:00, 48.86it/s] 


,activity_id,player_period,form_period,role_period,session,start_dt,end_dt,duration,player_id,squad_num,player_name,base_role,x,y
0,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1235,23,P23,8,-983.0,-1609.0
1,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1248,7,P07,3,910.0,1568.0
2,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1252,9,P09,5,1554.0,-577.0
3,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1759,8,P08,4,-171.0,-457.0
4,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1773,98,P98,10,1389.0,436.0
5,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1925,4,P04,2,-1362.0,686.0
6,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,4779,3,P03,1,-706.0,1597.0
7,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,5439,15,P15,7,-1294.0,-522.0
8,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,5713,10,P10,6,73.0,509.0
9,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,6088,72,P72,9,590.0,-1631.0


In [15]:
role_records_files = os.listdir(f'data/{mode}/role')
role_records_files.sort()
role_records_list = []

for file in role_records_files:
    path = f'data/{mode}/role/{file}'
    role_records_list.append(pd.read_csv(path, header=0))

role_records = pd.concat(role_records_list, ignore_index=True)
for col in HEADER_ROLE_RECORDS[:5]:
    role_records[col] = role_records[col].astype(int)

role_records.to_csv(f'data/{mode}/role_records.csv', index=False)
role_records

,activity_id,player_period,form_period,role_period,session,start_dt,end_dt,duration,player_id,squad_num,player_name,base_role,x,y
0,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1235,23,P23,8,-983.0,-1609.0
1,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1248,7,P07,3,910.0,1568.0
2,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1252,9,P09,5,1554.0,-577.0
3,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1759,8,P08,4,-171.0,-457.0
4,12864,1,1,1,1,2020-01-01 14:00:00,2020-01-01 14:47:00,2820,1773,98,P98,10,1389.0,436.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22265,9285,3,2,4,2,2020-01-01 15:31:00,2020-01-01 15:35:00,240,4219,14,P14,4,-38.0,-722.0
22266,9285,4,2,5,2,2020-01-01 15:35:00,2020-01-01 15:49:00,840,4219,14,P14,4,-38.0,-722.0
22267,9285,5,2,6,2,2020-01-01 15:49:00,2020-01-01 15:52:00,180,4219,14,P14,8,1003.0,1718.0
22268,9285,4,2,5,2,2020-01-01 15:35:00,2020-01-01 15:49:00,840,4740,7,P07,8,1003.0,1718.0
